In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os

import numpy as np
import pandas as pd
import sklearn
import torch
import tqdm

import celltrip


# Load Data and Policy

In [3]:
# Read data files
adata_prefix = 's3://nkalafut-celltrip/Flysta3D'
# adata_prefix = '../data/Flysta3D'
adatas = [
    celltrip.utility.processing.merge_adatas(
        *celltrip.utility.processing.read_adatas(*[
            f'{adata_prefix}/{p}_{m}.h5ad'
            for p in ('E14-16h_a', 'E16-18h_a', 'L1_a', 'L2_a', 'L3_b')
            # for p in ('L2_a',)
        ], backed=True), backed=True)
    for m in ('expression', 'spatial')]
# Model location and name (should be prefix for .weights, .pre, and .mask file)
# prefix, training_step = 's3://nkalafut-celltrip/checkpoints/flysta-250909-5', 800  # Double-standard
prefix, training_step = 's3://nkalafut-celltrip/checkpoints/flysta-250909-4', 800  # Regular
# prefix, training_step = 's3://nkalafut-celltrip/checkpoints/Flysta-251026', 800  # Additional hidden layer
# Generate or load preprocessing
preprocessing = celltrip.utility.processing.Preprocessing().load(f'{prefix}.pre')
with celltrip.utility.general.open_s3_or_local(f'{prefix}.mask', 'rb') as f:
    mask = np.loadtxt(f).astype(bool)
adatas[0].obs['Training'] = mask  # For meta export, note that obs is stored in memory
# Create sample env (kind of a dumb workaround, TODO)
m1, m2 = [preprocessing.transform(ad[:2].X, subset_modality=i)[0] for i, ad in enumerate(adatas)]
env = celltrip.environment.EnvironmentBase(
    torch.tensor(m1), torch.tensor(m2), target_modalities=[1], compute_rewards=False, dim=8).eval().to('cuda')
# Load policy
policy = celltrip.policy.create_agent_from_env(
    env, forward_batch_size=5_000, vision_size=1_000, pinning_spatial=[1]).eval().to('cuda')
policy.load_checkpoint(f'{prefix}-{training_step:04}.weights');


# Generate Steady States

In [4]:
# for dev in (pbar := tqdm.tqdm(adatas[0].obs['development'].unique(), desc='')):
#     # Subset and preprocess the data
#     pbar.set_description(f'{dev} (Preprocessing)')
#     samples = adatas[0].obs.index[adatas[0].obs['development']==dev]
#     # if len(samples) > 10_000: samples = np.random.choice(samples, 10_000, replace=False)  # For runtime, TESTING
#     m1, m2 = [
#         celltrip.utility.processing.chunk_X(
#             ad[samples], chunk_size=2_000,
#             func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
#             for i, ad in enumerate(adatas)]
#     # Initialize environment
#     pbar.set_description(f'{dev} (Initializing)')
#     env = celltrip.environment.EnvironmentBase(
#         torch.tensor(m1), torch.tensor(m2), target_modalities=[1], compute_rewards=False, dim=8).eval(time_scale=1).to('cuda')  # 32/env.max_time
#     # Simulate to steady state
#     pbar.set_description(f'{dev} (Running)')
#     # env.train().eval(time_scale=1)
#     env.reset()
#     ret = celltrip.train.simulate_until_completion(env, policy, skip_states=100, store_states='cpu')  # progress_bar=True
#     steady_state = ret[-1][-1, :, :env.dim]
#     target_state = env.modalities[env.target_modalities[0]].cpu()
#     with torch.no_grad():
#         imputed_steady_state = policy.pinning[0](steady_state.to('cuda'), Y=target_state.to('cuda')).detach().cpu().numpy()
#     imputed_steady_state, = preprocessing.inverse_transform(imputed_steady_state, subset_modality=1)
#     # Save
#     pbar.set_description(f'{dev} (Saving)')
#     np.save(f'../plots/flysta/CellTRIP_{dev}.npy', imputed_steady_state)
#     np.save(f'../plots/flysta/spatial_{dev}.npy', adatas[1][samples].X)
#     adatas[0].obs.loc[samples].to_csv(f'../plots/flysta/meta_{dev}.csv', index=False);


## Run Comparison Methods

In [5]:
# # Load full data
# X, Y = [
#     celltrip.utility.processing.chunk_X(
#         ad, chunk_size=2_000,
#         func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
#         for i, ad in enumerate(adatas)]


In [6]:
# # Train MLP and export predictions
# model = sklearn.neural_network.MLPRegressor(max_iter=100, verbose=True).fit(X[mask], Y[mask])
# for dev in (pbar := tqdm.tqdm(adatas[0].obs['development'].unique())):
#     # Subset and preprocess the data
#     pbar.set_description(f'{dev} (Preprocessing)')
#     samples = adatas[0].obs.index[adatas[0].obs['development']==dev]
#     X_dev, Y_dev = [
#         celltrip.utility.processing.chunk_X(
#             ad[samples], chunk_size=2_000,
#             func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
#             for i, ad in enumerate(adatas)]
#     # Run model
#     pbar.set_description(f'{dev} (Running)')
#     Y_pred = model.predict(X_dev)
#     imputed_steady_state, = preprocessing.inverse_transform(Y_pred, subset_modality=1)
#     # Save
#     pbar.set_description(f'{dev} (Saving)')
#     np.save(f'../plots/flysta/MLP_{dev}.npy', imputed_steady_state)


In [7]:
# # Export KNN predictions
# model = sklearn.neighbors.KNeighborsRegressor(n_neighbors=10).fit(X[mask], Y[mask])
# for dev in (pbar := tqdm.tqdm(adatas[0].obs['development'].unique())):
#     # Subset and preprocess the data
#     pbar.set_description(f'{dev} (Preprocessing)')
#     samples = adatas[0].obs.index[adatas[0].obs['development']==dev]
#     X_dev, Y_dev = [
#         celltrip.utility.processing.chunk_X(
#             ad[samples], chunk_size=2_000,
#             func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
#             for i, ad in enumerate(adatas)]
#     # Run model
#     pbar.set_description(f'{dev} (Running)')
#     Y_pred = model.predict(X_dev)
#     imputed_steady_state, = preprocessing.inverse_transform(Y_pred, subset_modality=1)
#     # Save
#     pbar.set_description(f'{dev} (Saving)')
#     np.save(f'../plots/flysta/KNN_{dev}.npy', imputed_steady_state)


# Recover Validation State

In [8]:
# # Separate training and validation stages
# development = np.array(['E14-16h_a', 'E16-18h_a', 'L1_a', 'L2_a', 'L3_b'])  # Ordered
# development_training = adatas[0].obs.loc[mask, 'development'].unique()
# development_validation = adatas[0].obs.loc[~mask, 'development'].unique()
# assert len(np.intersect1d(development_training, development_validation)) == 0  # Properly partitioned
# # Get possible interpolation stages
# possible_interpolated_stages = []
# for i in np.argwhere(np.isin(development, development_validation)).flatten():
#     if i == 0 or i == len(development)-1: continue
#     possible_interpolated_stages.append(development[i-1:i+2])
# # Set interpolation
# start_stage, interp_stage, end_stage = possible_interpolated_stages[-1]


In [9]:
# # Grab data
# start_idx = np.argwhere(adatas[0].obs['development'] == start_stage).flatten()
# end_idx = np.argwhere(adatas[0].obs['development'] == end_stage).flatten()
# start_exp = celltrip.utility.processing.chunk_X(
#     adatas[0][start_idx], chunk_size=2_000,
#     func=lambda x: preprocessing.transform(x, subset_modality=0)[0])
# end_exp = celltrip.utility.processing.chunk_X(
#     adatas[0][end_idx], chunk_size=2_000,
#     func=lambda x: preprocessing.transform(x, subset_modality=0)[0])
# start_obs = celltrip.utility.general.transform_and_center(celltrip.utility.processing.chunk_X(adatas[1][start_idx], chunk_size=2_000))
# end_obs = celltrip.utility.general.transform_and_center(celltrip.utility.processing.chunk_X(adatas[1][end_idx], chunk_size=2_000))

# # Use K-Means to create start and end pseudocells
# start_n_pcells = end_n_pcells = 5_000
# start_pcell_ids = sklearn.cluster.KMeans(n_clusters=start_n_pcells, random_state=42).fit_predict(start_obs)
# end_pcell_ids = sklearn.cluster.KMeans(n_clusters=end_n_pcells, random_state=42).fit_predict(start_obs)

# # Get expression and spatial for pseudocells
# start_processed_exp = np.stack([start_exp[np.argwhere(start_pcell_ids==i).flatten()].mean(axis=0) for i in range(start_n_pcells)], axis=0)
# start_processed_obs = np.stack([start_obs[np.argwhere(start_pcell_ids==i).flatten()].mean(axis=0) for i in range(start_n_pcells)], axis=0)
# end_processed_exp = np.stack([end_exp[np.argwhere(end_pcell_ids==i).flatten()].mean(axis=0) for i in range(end_n_pcells)], axis=0)
# end_processed_obs = np.stack([end_obs[np.argwhere(end_pcell_ids==i).flatten()].mean(axis=0) for i in range(end_n_pcells)], axis=0)

# # Calculate OT matrix
# a, b, _, OT_mat = celltrip.utility.general.compute_discrete_ot_matrix(start_processed_obs, end_processed_obs)

# # Calculate pseudocells
# pcells = [([i], np.argwhere(OT_mat[i] > 0).flatten()) for i in range(OT_mat.shape[0]) if OT_mat[i].sum() > 0]
# start_pcells_exp, end_pcells_exp = [], []
# start_pcells_obs, end_pcells_obs = [], []
# for pcell_start, pcell_end in pcells:
#     start_pcells_exp.append(start_processed_exp[pcell_start].mean(axis=0))
#     end_pcells_exp.append(end_processed_exp[pcell_end].mean(axis=0))
#     start_pcells_obs.append(start_processed_obs[pcell_start].mean(axis=0))
#     end_pcells_obs.append(end_processed_obs[pcell_end].mean(axis=0))
# start_pcells_exp = np.stack(start_pcells_exp, axis=0)
# end_pcells_exp = np.stack(end_pcells_exp, axis=0)
# start_pcells_obs = np.stack(start_pcells_obs, axis=0)
# end_pcells_obs = np.stack(end_pcells_obs, axis=0)


In [10]:
# # Create env
# m1_start, m1_end = start_pcells_exp, end_pcells_exp  # preprocessing.transform
# env = celltrip.environment.EnvironmentBase(
#     torch.tensor(m1_start), target_modalities=None, compute_rewards=False, dim=8).eval(time_scale=1).to('cuda')

# # Get transition states
# env.reset()
# celltrip.train.simulate_until_completion(env, policy, store_states=False)  # Set env at steady state
# env.time = 0  # Reset timing
# env.set_modalities([torch.tensor(m1_end)]).to('cuda')  # Set to ending expression
# transition_states = celltrip.train.simulate_until_completion(env, policy, skip_states=50, store_states='cpu', progress_bar=True)[-1][..., :env.dim].cpu()

# # Impute transition states
# with torch.no_grad():
#     imputed_transition_states = policy.pinning[0](transition_states.to('cuda')).detach().cpu().numpy()
# imputed_transition_states, = preprocessing.inverse_transform(imputed_transition_states, subset_modality=1)
# np.save(f'../plots/flysta/Interpolated_CellTRIP_{interp_stage}.npy', imputed_transition_states)


## Run Comparison Methods

In [11]:
# # Export LERP
# for progress in (.25, .5, .75):
#     imputed_transition_states = ((1-progress)*start_pcells_obs + progress*end_pcells_obs) / 2
#     np.save(f'../plots/flysta/Interpolated_LERP-{progress:.2f}_{interp_stage}.npy', imputed_transition_states)


# Perform Knockdown

In [12]:
rep_sample = adatas[0][np.random.choice(adatas[0].shape[0], 1_000, replace=False)].X

In [13]:
(rep_sample.mean(axis=0) == 0).sum()

np.int64(694)

In [14]:
# Params
np.random.seed(42)
# genes_to_survey = np.random.choice(adatas[0].var_names, 2000, replace=False)
# assert preprocessing.filter_mask[0] is None
genes_to_survey = adatas[0].var_names[preprocessing.standardize_std[0].flatten().argsort()[::-1]][:1000]  # 2k hvg
# genes_to_survey = adatas[0].var_names
sim_time = .5

# Add results
results = []
def add_record(states, gene, dev, ct):
    results.append({
        'Gene': gene, 'Development': dev, 'Cell Type': ct,
        'Effect Size': np.sqrt(np.square(states[-1] - states[0]).sum(axis=-1)).mean(),
        'Trajectory Length': np.sqrt(np.square(states[1:] - states[:-1])).mean(axis=-1).sum()})
    
# Reset function
def reset_env(env, steady_pos, steady_vel, modal_dict={}):
    env.set_max_time(sim_time).reset()  # TODO: Maybe longer?, early stopping?
    env.set_positions(steady_pos)
    env.set_velocities(steady_vel)  # Maybe 0 manually?
    for k, v in modal_dict.items():
        env.modalities[k] = v

# Running function
def run_and_record(samples, env, policy, preprocessing, gene, dev, gene_idx):
    # Run and impute
    states = celltrip.train.simulate_until_completion(
        env, policy,
        env_hooks=[
            celltrip.utility.hooks.clamp_inverted_features_hook(
                gene_idx, preprocessing, feature_targets=0., modality_idx=0),
        ],
        store_states='cpu')[-1]
    with torch.no_grad():
        imputed_states = policy.pinning[0](states[..., :env.dim].to('cuda')).detach().cpu().numpy()
    imputed_states, = preprocessing.inverse_transform(imputed_states, subset_modality=1)
    # Record
    add_record(imputed_states, gene, dev, 'All')
    for ct in adatas[0][samples].obs['annotation'].unique():
        add_record(imputed_states[:, adatas[0][samples].obs['annotation']==ct], gene, dev, ct)

# Evaluate genes
for dev in adatas[0].obs['development'].unique():
    # Subset and preprocess the data
    samples = adatas[0].obs.index[adatas[0].obs['development']==dev]
    raw_m1 = celltrip.utility.processing.chunk_X(adatas[0][samples], chunk_size=2_000)
    m1, m2 = [
        celltrip.utility.processing.chunk_X(
            ad[samples], chunk_size=2_000,
            func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
            for i, ad in enumerate(adatas)]
    
    # Initialize environment
    env = celltrip.environment.EnvironmentBase(
        torch.tensor(m1), torch.tensor(m2), target_modalities=[1], compute_rewards=False, dim=8).eval(time_scale=5).to('cuda')
    
    # Simulate to steady state
    env.reset()
    celltrip.train.simulate_until_completion(env, policy)
    steady_pos, steady_vel = (env.pos, env.vel)

    # Run control
    reset_env(env, steady_pos, steady_vel)
    run_and_record(samples, env, policy, preprocessing, 'Control', dev, [])

    # Perturb
    for gene in tqdm.tqdm(genes_to_survey, desc=dev, miniters=10, maxinterval=30):
        # Get gene idx and reset environment
        gene_idx = np.argwhere(adatas[0].var_names==gene).flatten()
        reset_env(env, steady_pos, steady_vel)  # {0: torch.tensor(m1).cuda()}

        # Apply knockdown
        # # iso_modality, = preprocessing.transform(raw_m1, subset_features=gene_idx, subset_modality=0)
        # iso_modality = celltrip.utility.processing.chunk(
        #     raw_m1, chunk_size=2_000, func=lambda x: preprocessing.transform(x, subset_features=gene_idx, subset_modality=0)[0])
        # iso_modality = torch.tensor(iso_modality).to(env.device)
        # modal_change = (iso_modality - 0*iso_modality)  # Coeff is for overexp anal, NOTE: if `pre_log`, this method will not work for overexp
        # env.modalities[0] = env.modalities[0] - modal_change

        # Simulate and record
        run_and_record(samples, env, policy, preprocessing, gene, dev, gene_idx)
        # env.modalities[0] = env.modalities[0] + modal_change

# Convert and save
pd.DataFrame(results).to_csv('../plots/flysta/knockdown.csv', index=None)

# .5s run on 20 genes (opposite order)
# E14-16h_a: 100%|██████████| 20/20 [01:02<00:00,  3.11s/it]
# E16-18h_a: 100%|██████████| 20/20 [00:56<00:00,  2.84s/it]
# L1_a: 100%|██████████| 20/20 [01:08<00:00,  3.41s/it]
# L2_a: 100%|██████████| 20/20 [04:32<00:00, 13.61s/it]
# L3_b: 100%|██████████| 20/20 [02:55<00:00,  8.78s/it]

# 1s run on 20 genes (40m)
# E14-16h_a: 100%|██████████| 20/20 [01:47<00:00,  5.36s/it]
# E16-18h_a: 100%|██████████| 20/20 [01:40<00:00,  5.05s/it]
# L1_a: 100%|██████████| 20/20 [01:54<00:00,  5.72s/it]
# L2_a: 100%|██████████| 20/20 [07:43<00:00, 23.16s/it]
# L3_b: 100%|██████████| 20/20 [05:04<00:00, 15.21s/it]


E14-16h_a:   0%|                                                                                                                                                           | 0/1000 [00:00<?, ?it/s]

E14-16h_a:   1%|█▍                                                                                                                                                | 10/1000 [00:12<21:08,  1.28s/it]

E14-16h_a:   2%|██▉                                                                                                                                               | 20/1000 [00:25<20:52,  1.28s/it]

E14-16h_a:   3%|████▍                                                                                                                                             | 30/1000 [00:38<20:27,  1.27s/it]

E14-16h_a:   4%|█████▊                                                                                                                                            | 40/1000 [00:50<20:22,  1.27s/it]

...

L3_b: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 997/1000 [1:04:40<00:11,  3.90s/it]

L3_b: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 998/1000 [1:04:44<00:07,  3.87s/it]

L3_b: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 999/1000 [1:04:48<00:03,  3.85s/it]

L3_b: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:04:52<00:00,  3.87s/it]

L3_b: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:04:52<00:00,  3.89s/it]